# Imports

In [53]:
import pandas as pd
# import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime as dt
import requests
import random
import json
import csv
import time
import folium
import os
from folium import Map, CircleMarker, Vega, Popup, Marker, PolyLine, Icon
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.features import ColorLine
from math import radians, sin, cos, asin, sqrt
from urllib.parse import urljoin

from math import radians, sin, cos, asin, sqrt
# from ipywidgets import interact

# Data

In [2]:
data_senior = pd.read_csv('raw_data/senior_transportation.csv')
data_bus = pd.read_csv('raw_data/bus_routes.csv')
data_users = pd.read_csv('raw_data/churn_users.csv')
data_od = pd.read_csv('raw_data/churn_OD.csv')

In [3]:
data_senior

,linkid,Region_of_Origin,District_of_Origin,County_of_Origin,Average_Daily_SeniorPopulation_Travelling
0,80216810,R1 - AM Lisboa,Lisboa,Lisboa,2013.681187
1,80216810,R1 - AM Lisboa,Setúbal,Palmela,7688.296396
2,80216810,R1 - AM Lisboa,Lisboa,Oeiras,220.647248
3,80216810,R1 - AM Lisboa,Lisboa,Vila Franca de Xira,233.203075
4,80216812,R1 - AM Lisboa,Lisboa,Lisboa,261.606014
...,...,...,...,...,...
121048,1223824210,R1 - AM Lisboa,Setúbal,Barreiro,1877.243984
121049,1223824210,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121050,1223824211,R1 - AM Lisboa,Setúbal,Palmela,505.273329
121051,1223824211,R1 - AM Lisboa,Setúbal,Barreiro,961.962684


# HERE API

In [59]:
urljoin(base_url,endpoint_index)

'https://fleet.ls.hereapi.com/index.json'

In [58]:
base_url = 'https://fleet.ls.hereapi.com/1'
endpoint_index = '/index.json'
params_i = {'layer':'ROAD_GEOM_FCn',
          'attributes':'LINK_ID',
          'values':'1223824210',
          'apiKey':'boaGJdln4BbKpJVXu7c1CBUupySsaKWWclGWvM-tJXw'}
ri = requests.get(url=urljoin(base_url,endpoint_index), params=params_i)
ri

<Response [404]>

In [54]:
def get_tiles(link_id):
    ## going yo index API
    base_url = 'https://fleet.ls.hereapi.com/1'
    endpoint_index = '/index.json'
    params_i = {'layer':'ROAD_GEOM_FCn',
              'attributes':'LINK_ID',
              'values':f'{link_id}',
              'apiKey':'boaGJdln4BbKpJVXu7c1CBUupySsaKWWclGWvM-tJXw'}
    ri = requests.get(url=urljoin(base_url,endpoint_index), params=params_i)
    layers = ri.json().get('Layers')[0]
    layer = layers.get('layer', None)
    level = layers.get('level', None)
    tileXYs = layers.get('tileXYs', None)[0]
    
    ## going to tile API
    endpoint_tile = '/tile.json'
    params_t = {'apiKey':'boaGJdln4BbKpJVXu7c1CBUupySsaKWWclGWvM-tJXw',
            'layer':layer,
            'level':level,
            'tilex':tileXYs.get('x'),
            'tiley':tileXYs.get('y')}
    rt = requests.get(url=urljoin(base_url,endpoint_tile), params=params_t)
    response = rt.json()
    
    coords = []
    for point in response.get('Rows'):
        lat = float(point.get('LAT')[:point.get('LAT').find(",")].replace(",","."))/100000
        lon = float(point.get('LON')[:point.get('LON').find(",")].replace(",","."))/100000
        coords.append((lat, lon))
    return coords

In [56]:
for c, row in data_senior.head(20).iterrows():
    row['coords'] = get_tiles(row['linkid'])

data_senior

TypeError: 'NoneType' object is not subscriptable

In [33]:
coords = []
for point in response.get('Rows'):
    lat = float(point.get('LAT')[:point.get('LAT').find(",")].replace(",","."))/100000
    lon = float(point.get('LON')[:point.get('LON').find(",")].replace(",","."))/100000
    coords.append((lat, lon))

In [34]:
m = folium.Map([38.668335400000004, -9.0533746])
for coord in coords:
    Marker(coord).add_to(m)
m